<a href="https://colab.research.google.com/github/risherb/fantasy-league/blob/main/fantasy_premier_league.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'fantasy-premier-league-fpl-player-data-2024-2025:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5686253%2F9468317%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241004%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241004T055408Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D303d66589d36e740bc71359167a5f35699ec882d7a007f9a6278bf5de2d34aa69627d483f02f2272b1fa59431a6f15383394b20c182f2b048aeae8db00e6b0e0baa7f9da588efb6fd4e5320a3197c184564a5ffb335cec7aa606032fb543d4f9bcb061aea3ebade819201dbaf1ae1add6b432142ef2f924256962c653f07b4945a9abac13a315386a3c17647ed510e6a410947effcf17eebe6e307cb5e92b1082c7af9bde0342d47d07204d2edadbb186752bf39e0c4bec491768b4ac5a63524eb93f2113e47a5df1c916780c7808d1099d118cc46f99c815cb46dea2ccc7a07ac5ea765afadf699880516abc887633b0479f4a953fdf2c8c883db03a68fcf3e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
df = pd.read_csv("/kaggle/input/fantasy-premier-league-fpl-player-data-2024-2025/fpl_playerstats_2024-25.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

## missing values

In [ ]:
df.isna().sum()

In [ ]:
df.drop(["team_form"], inplace =True ,axis = 1  )
df.sample(10)

In [ ]:
x= df['gw1_points'].mode()[0]
df['gw1_points'].fillna(x , inplace = True)
df['gw2_points'].fillna(x , inplace = True)
df['gw3_points'].fillna(x , inplace = True)
df['gw4_points'].fillna(x , inplace = True)

In [ ]:
df.isna().sum()

In [ ]:
df.head()

In [ ]:
fig = px.bar(df, x='player_position',  y= 'total_points' , color = 'team_name', title = 'max position point')
fig.show()

In [ ]:
fig = px.pie(df, names ='team_name',  values= 'total_points' , title = 'max team points')
fig.show()

In [ ]:
fig = px.bar(df, x='team_name',  y= 'total_points' , color = 'player_position', title = 'Best positions per team')
fig.show()

In [ ]:
fig = px.bar(df, x='player_position',  y= 'goals_scored' , color = 'assists', title = 'most position Contribution')
fig.show()

In [ ]:
fig = px.bar(df, x='team_name',  y= 'yellow_cards' , color = 'red_cards', title = 'most cards per team')
fig.show()

In [ ]:
fig = px.bar(df, x='player_cost',  y= 'goals_scored' , color = 'second_name', title = 'Most effective')
fig.show()